In [ ]:
!pip freeze 

In [ ]:
!pip install kserve==0.9

In [1]:
import kserve

In [ ]:
kserve.Model??

In [2]:
import kserve
from typing import Dict, List
import argparse
import urllib
import numpy as np
import PIL.Image as Image
from urllib import request
from io import BytesIO


class ImageTransformer(kserve.Model):
    def __init__(self, name: str, predictor_host: str):
        super().__init__(name)
        self.predictor_host = predictor_host
        labels_path="./ImageNetLabels.txt"
        imagenet_labels = np.array(open(labels_path).read().splitlines())
        self.imagenet_labels = imagenet_labels
        self.IMAGE_SHAPE = (224, 224)
        self.ready = True
        
    def image_transform(self, instance) -> np.array:
        image_url = instance["url"]
        res = request.urlopen(image_url).read()
        image_data_rs = Image.open(BytesIO(res)).resize(self.IMAGE_SHAPE)
        image_data_rs_n = np.array(image_data_rs)/255.0
        X_data = image_data_rs_n.tolist()
        return X_data
        
    def preprocess(self, inputs: Dict) -> Dict:
        instance = {'instances': [self.image_transform(instance) for instance in inputs['instances']]}
        return instance

    def postprocess(self, model_output_data: Dict) -> Dict:
        pred = model_output_data.json()['predictions'][0]
        predicted_class = np.argmax(pred, axis=-1)
        predicted_class_name = self.imagenet_labels[predicted_class]
        return {"predictions" : predicted_class_name}

In [3]:
it=ImageTransformer("model","host")

In [4]:
image_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/grace_hopper.jpg"
data = {
    "instances": [
        {"url": image_url},
    ]
}

In [5]:
preprocess_data = it.preprocess(data)

In [6]:
from kserve import (
    KServeClient,
    utils
)
from kubernetes import client

service_name = "mobilenet-v2"
namespace = utils.get_default_target_namespace()

KServe = KServeClient()

KServe.get(service_name, namespace=namespace, watch=True, timeout_seconds=120)
isvc_resp=KServe.get(service_name, namespace=namespace)

import requests
isvc_url = isvc_resp['status']['address']['url']
print(isvc_url)

NAME                 READY                           PREV                    LATEST URL                                                              
mobilenet-v2         True                               0                       100 http://mobilenet-v2.kubeflow-user-example-com.example.com        
http://mobilenet-v2.kubeflow-user-example-com.svc.cluster.local/v1/models/mobilenet-v2:predict


In [7]:
response = requests.post(isvc_url, json=preprocess_data)
print(response.text)

{
    "predictions": [[0.199675947, 1.0678966, -0.096099332, 1.10753763, -0.154607415, -0.631894469, -0.0360174924, -2.01866984, -0.14397347, -0.143937051, -0.296572417, 0.527685523, 0.583141685, -0.810139537, -0.65175426, 0.926327348, 0.00984019, -1.29104185, 1.16821468, 0.153163046, -0.587534726, 0.0997167304, 0.238366321, 1.06880593, -0.694764137, -0.4509179, 0.151563928, -1.01612639, 0.564389467, 0.173862651, -1.75571036, -0.286650717, 0.393435389, 0.523286462, -0.768849373, -1.38230455, -1.22526884, -1.91696084, -0.829766273, -0.323191166, 0.0605888963, -0.756531417, 0.404557735, -0.411388397, -1.16386867, -0.185540974, 0.599889517, -1.12346721, -0.142629325, -1.48209381, 0.737982571, 0.118609346, -0.433812171, -1.16172123, 0.463743031, -0.898387134, -0.136469662, 0.106437027, 0.00677442551, 0.260293156, -0.082685709, -0.171738356, -0.745466709, 0.0606884286, -0.662643969, 0.371543288, 0.606449, -0.11554873, -0.456262529, -0.561569691, 0.322999179, -0.192452818, -0.270121366, -0.0

In [8]:
it.postprocess(response)

{'predictions': 'military uniform'}

#

In [11]:
from kserve import (
    KServeClient,
    utils
)
from kubernetes import client

service_name = "mobilenet-v2-transformer"
namespace = utils.get_default_target_namespace()

KServe = KServeClient()

KServe.get(service_name, namespace=namespace, watch=True, timeout_seconds=120)
isvc_resp=KServe.get(service_name, namespace=namespace)

NAME                 READY                           PREV                    LATEST URL                                                              
mobilenet-v2-tran... True                               0                       100 http://mobilenet-v2-transformer.kubeflow-user-example-com.exam...


In [12]:
import requests
isvc_url = isvc_resp['status']['address']['url']
print(isvc_url)

response = requests.post(isvc_url, json=data)
print(response.text)

http://mobilenet-v2-transformer.kubeflow-user-example-com.svc.cluster.local/v1/models/mobilenet-v2-transformer:predict
{"predictions": "military uniform"}
